## Cert-Manager Installation

Dazu greifen wir auf Scripts aus dem [Lern Cloud Projekt](https://github.com/mc-b/lerncloud/tree/main/services) zurück:
* Installiert den Cert-Manager
* Richte eine CA für interne Zertifikate ein.


In [ ]:
%%bash
curl -sfL https://raw.githubusercontent.com/mc-b/lerncloud/main/services/cert-manager.sh | bash -

### Zertifikatsanfrage für die interne Domain erstellen

Wir erstellen pro Kubernetes Namespace ein Zertifikat.

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: cert-manager.io/v1
kind: Certificate
metadata:
  name: selfsigned-cert-ch
  namespace: m01-ch
spec:
  secretName: selfsigned-cert-secret
  duration: 2400h
  renewBefore: 12h
  issuerRef:
    name: selfsigned-cluster-issuer
    kind: ClusterIssuer
  commonName: ch.$(cat ~/work/server-ip)-edutbz.com
  dnsNames:
    - ch.$(cat ~/work/server-ip)-edutbz.com
EOF

- - -

## Testen

Das Zertifikat liegt nun als Kubernetes-Secret unter `selfsigned-cert-secret` und kann z. B. für Ingress oder TLS-Services genutzt werden.

Wir Testen es mit dem Auto Shop Webshop, welcher vorgängig installiert werden muss.

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: webshop 
  namespace: m01-ch
  annotations:
    cert-manager.io/issuer: selfsigned-cluster-issuer
spec:
  rules:
  - host: ch.$(cat ~/work/server-ip)-edutbz.com
    http:
      paths:
      - path: /webshop
        pathType: Prefix
        backend:
          service:
            name: webshop
            port:
              number: 8080
  tls:
  - hosts:
    - ch.$(cat ~/work/server-ip)-edutbz.com
    secretName: selfsigned-cert-secret
EOF

### Kontrolle des Zertifikats
Testen, ob das Zertifikat übernommen wurde.

In [ ]:
%%bash
helm status autoshop-ch --namespace m01-ch

Anschliessend die URL im Browser aufrufen und die Zertifikatsdetails prüfen. Der allgemeine Name (CN) sollte `ch.$(cat ~/work/server-ip)-edutbz.com` sein.

- - -

## Aufträge

* Vervollständig das Beispiel durch die Erstellung der Zertifikate für die anderen K8s Namespaces at und de.
* Ersetzt die internen Zertifikate durch von einer CA validierte Zertifikate, z.B. mittels [Let's Encrypt](https://letsencrypt.org/de/).